
### 1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>. You have to use data.csv file for this assignment
### 2. Code the model to classify data like below image. You can use any number of units in your Dense layers.

<img src='https://i.imgur.com/33ptOFy.png'>



# <font color='red'> <b>3. Writing Callbacks </b> </font>
## You have to implement the following callbacks
-  Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.Do not use tf.keras.metrics for calculating AUC and F1 score.

- Save your model at every epoch if your validation accuracy is improved from previous epoch. 

- You have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
- If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

- You have to stop the training if your validation accuracy is not increased in last 2 epochs.

- Use tensorboard for every model and analyse your scalar plots and histograms. (you need to upload the screenshots and write the observations for each model for evaluation)



In [1]:
# !gdown 15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM

In [2]:
# Clear any logs from previous runs
!rm -rf ./N_logs/

In [3]:
import datetime
import numpy as np
import pandas as pd
# import tensorflow as tf

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation

from tensorflow.keras.initializers import HeUniform
from tensorflow.keras.initializers import GlorotNormal
from tensorflow.keras.initializers import RandomUniform

from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

%load_ext tensorboard

2022-03-05 16:19:30.742067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-05 16:19:30.742121: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
# https://www.tensorflow.org/tutorials/load_data/csv

data = pd.read_csv('data.csv')

print(f'Shape of data set : {data.shape}')
# pd.read_csv('data.csv').head()

Shape of data set : (20000, 3)


In [5]:
# Checking how the data is balanced !

data.label.value_counts()

0.0    10000
1.0    10000
Name: label, dtype: int64

In [6]:
# Splitting data into train-test

y_label = data.label
x_values = data.drop(['label'], axis = 1)

x_train, x_test, y_train, y_test = train_test_split(x_values, y_label, test_size = 0.3, random_state = 5)

**<font color ='red'>Custom Callbacks</font>**

In [7]:
class roc_score(Callback):
    
    def __init__(self, train_x, test_x, train_y, test_y):
        
        self.x_tr = train_x
        self.y_tr = train_y
        self.x_te = test_x
        self.y_te = test_y
        
    def on_epoch_end(self, epoch, logs = {}):

        y_pred_tr = self.model.predict(self.x_tr)
        y_pred_te = self.model.predict(self.x_te)
            
        roc_tr = roc_auc_score(self.y_tr, y_pred_tr)
        roc_te = roc_auc_score(self.y_te, y_pred_te)
        
        print(f' ROC_test:{round(roc_te, 3)}')

class f1_score_(Callback):
    
    def __init__(self, train_x, test_x, train_y, test_y):
        
        self.x_tr = train_x
        self.y_tr = train_y
        self.x_te = test_x
        self.y_te = test_y
        
    def on_epoch_end(self, epoch, logs = {}):

        y_pred_tr = (np.array(self.model.predict(self.x_tr))).round()
        y_pred_te = (np.array(self.model.predict(self.x_te))).round()
            
        f1_tr = f1_score(self.y_tr, y_pred_tr, average = 'micro')
        f1_te = f1_score(self.y_te, y_pred_te, average = 'micro')
        
        print(f' MicroF1_Test:{round(f1_te, 3)}')

        
class TerminateNaN(Callback):

    def on_epoch_end(self, epoch, logs = {}):
        
        loss = logs.get('loss')
        
        if loss is not None:
            
            if np.isnan(loss) or np.isinf(loss):
                print(F'Invalid loss and terminated at epoch {epoch}')
                self.model.stop_training = True
                
                
roc_score = roc_score(x_train, x_test, y_train, y_test)

f1_score_ = f1_score_(x_train, x_test, y_train, y_test)

terminateNaN = TerminateNaN()

In [8]:
# https://keras.io/api/callbacks/model_checkpoint/
# https://keras.io/api/callbacks/early_stopping/
# https://keras.io/api/callbacks/reduce_lr_on_plateau/

early_stopping = EarlyStopping(monitor = 'val_accuracy', patience = 2, verbose = 1)

filpath = 'me_model_save/epo_{epoch:02d}-accu_{val_accuracy:.4f}.hdf5'
model_check_point = ModelCheckpoint(filpath, monitor = 'val_accuracy', patience = 1, verbose = 1)


# factor: factor by which the learning rate will be reduced. new_lr = lr * factor
# lr = 0.1, factor = 0.9.
# Then, new_lr = 0.09 ==> 10% reduction

epoch_1 = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.9, verbose = 1, patience = 1) # 10% reduction
epoch_3 = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.95, verbose = 1, patience = 3) # 5% reduction

In [9]:
def create_model(activ, initializer):
    
    model = Sequential()
    model.add(Dense(32, input_shape = (2,), kernel_initializer = initializer, name = 'input_layer'))
    model.add(Dense(32, activation = activ, kernel_initializer = initializer, name = 'dense1'))
    model.add(Dense(32, activation = activ, kernel_initializer = initializer, name = 'dense2'))
    model.add(Dense(32, activation = activ, kernel_initializer = initializer, name = 'dense3'))
    model.add(Dense(32, activation = activ, kernel_initializer = initializer, name = 'dense4'))
    model.add(Dense(32, activation = activ, kernel_initializer = initializer, name = 'dense5'))
    model.add(Dense(1, activation = 'sigmoid', name = 'output_layer'))

    model.summary()
    
    return model

EPOCH = 20

<br>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>

In [10]:
kernel_init = RandomUniform(0, 1)

model = create_model('tanh', kernel_init)

optimizer_ = optimizers.SGD(learning_rate = 0.001, momentum = 0.9, name = 'SGD')

model.compile(optimizer = optimizer_, loss = 'binary_crossentropy', metrics = ['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (Dense)         (None, 32)                96        
                                                                 
 dense1 (Dense)              (None, 32)                1056      
                                                                 
 dense2 (Dense)              (None, 32)                1056      
                                                                 
 dense3 (Dense)              (None, 32)                1056      
                                                                 
 dense4 (Dense)              (None, 32)                1056      
                                                                 
 dense5 (Dense)              (None, 32)                1056      
                                                                 
 output_layer (Dense)        (None, 1)                 3

2022-03-05 16:19:32.506025: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-05 16:19:32.506055: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-05 16:19:32.506072: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu): /proc/driver/nvidia/version does not exist
2022-03-05 16:19:32.506732: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# TensorBoard

log_dir = 'N_logs/1_tanh_Random'
tensorboard_ = TensorBoard(log_dir = log_dir, histogram_freq = 1, write_graph = True)

CallBack_list = [roc_score, f1_score_, terminateNaN, early_stopping, model_check_point,
                                                                     epoch_1, epoch_3, tensorboard_]

In [12]:
history = model.fit(x = x_train, y = y_train, epochs = EPOCH, verbose = 1,
            validation_data = (x_test, y_test), callbacks = CallBack_list)

Epoch 1/20
425/438 [============================>.] - ETA: 0s - loss: 0.6936 - accuracy: 0.4993 ROC_test:0.496
 MicroF1_Test:0.496

Epoch 1: saving model to me_model_save/epo_01-accu_0.4960.hdf5
438/438 [==============================] - 3s 6ms/step - loss: 0.6935 - accuracy: 0.4995 - val_loss: 0.6936 - val_accuracy: 0.4960 - lr: 0.0010
Epoch 2/20
429/438 [============================>.] - ETA: 0s - loss: 0.6939 - accuracy: 0.4927 ROC_test:0.496
 MicroF1_Test:0.496

Epoch 2: saving model to me_model_save/epo_02-accu_0.4960.hdf5

Epoch 2: ReduceLROnPlateau reducing learning rate to 0.0009000000427477062.
438/438 [==============================] - 3s 6ms/step - loss: 0.6939 - accuracy: 0.4918 - val_loss: 0.6933 - val_accuracy: 0.4960 - lr: 9.0000e-04
Epoch 3/20
414/438 [===========================>..] - ETA: 0s - loss: 0.6937 - accuracy: 0.4973 ROC_test:0.504
 MicroF1_Test:0.504

Epoch 3: saving model to me_model_save/epo_03-accu_0.5040.hdf5
438/438 [==============================] - 2s 

In [13]:
%tensorboard --logdir N_logs/

<br>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>

In [14]:
kernel_init = RandomUniform(0, 1)

model = create_model('relu', kernel_init)

optimizer_ = optimizers.SGD(learning_rate = 0.001, momentum = 0.9, name = 'SGD')

model.compile(optimizer = optimizer_, loss = 'binary_crossentropy', metrics = ['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (Dense)         (None, 32)                96        
                                                                 
 dense1 (Dense)              (None, 32)                1056      
                                                                 
 dense2 (Dense)              (None, 32)                1056      
                                                                 
 dense3 (Dense)              (None, 32)                1056      
                                                                 
 dense4 (Dense)              (None, 32)                1056      
                                                                 
 dense5 (Dense)              (None, 32)                1056      
                                                                 
 output_layer (Dense)        (None, 1)                

In [15]:
# TensorBoard

log_dir = 'N_logs/2_relu_Random'
tensorboard_ = TensorBoard(log_dir = log_dir, histogram_freq = 1, write_graph = True)

CallBack_list = [roc_score, f1_score_, terminateNaN, early_stopping, model_check_point,
                                                                     epoch_1, epoch_3, tensorboard_]

In [16]:
history = model.fit(x = x_train, y = y_train, epochs = EPOCH, verbose = 1,
            validation_data = (x_test, y_test), callbacks = CallBack_list)

Epoch 1/20
432/438 [============================>.] - ETA: 0s - loss: 1213.5258 - accuracy: 0.4941 ROC_test:0.5
 MicroF1_Test:0.494

Epoch 1: saving model to me_model_save/epo_01-accu_0.4942.hdf5
438/438 [==============================] - 3s 7ms/step - loss: 1198.2787 - accuracy: 0.4941 - val_loss: 0.6932 - val_accuracy: 0.4942 - lr: 0.0010
Epoch 2/20
421/438 [===========================>..] - ETA: 0s - loss: 0.6932 - accuracy: 0.5029 ROC_test:0.5
 MicroF1_Test:0.494

Epoch 2: saving model to me_model_save/epo_02-accu_0.4942.hdf5

Epoch 2: ReduceLROnPlateau reducing learning rate to 0.0009000000427477062.
438/438 [==============================] - 2s 5ms/step - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4942 - lr: 9.0000e-04
Epoch 3/20
430/438 [============================>.] - ETA: 0s - loss: 0.6932 - accuracy: 0.5008 ROC_test:0.5
 MicroF1_Test:0.494

Epoch 3: saving model to me_model_save/epo_03-accu_0.4942.hdf5

Epoch 3: ReduceLROnPlateau reducing learning 

In [17]:
%tensorboard --logdir N_logs/

Reusing TensorBoard on port 6006 (pid 3916), started 0:00:09 ago. (Use '!kill 3916' to kill it.)

<br>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>

In [18]:
kernel_init = HeUniform()

model = create_model('relu', kernel_init)

optimizer_ = optimizers.SGD(learning_rate = 0.001, momentum = 0.9, name = 'SGD')

model.compile(optimizer = optimizer_, loss = 'binary_crossentropy', metrics = ['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (Dense)         (None, 32)                96        
                                                                 
 dense1 (Dense)              (None, 32)                1056      
                                                                 
 dense2 (Dense)              (None, 32)                1056      
                                                                 
 dense3 (Dense)              (None, 32)                1056      
                                                                 
 dense4 (Dense)              (None, 32)                1056      
                                                                 
 dense5 (Dense)              (None, 32)                1056      
                                                                 
 output_layer (Dense)        (None, 1)                

In [19]:
# TensorBoard

log_dir = 'N_logs/3_relu_HeUniform'
tensorboard_ = TensorBoard(log_dir = log_dir, histogram_freq = 1, write_graph = True)

CallBack_list = [roc_score, f1_score_, terminateNaN, early_stopping, model_check_point,
                                                                     epoch_1, epoch_3, tensorboard_]

In [20]:
history = model.fit(x = x_train, y = y_train, epochs = EPOCH, verbose = 1,
            validation_data = (x_test, y_test), callbacks = CallBack_list)

Epoch 1/20
415/438 [===========================>..] - ETA: 0s - loss: 0.6914 - accuracy: 0.5336 ROC_test:0.705
 MicroF1_Test:0.574

Epoch 1: saving model to me_model_save/epo_01-accu_0.5740.hdf5
438/438 [==============================] - 3s 6ms/step - loss: 0.6904 - accuracy: 0.5369 - val_loss: 0.6698 - val_accuracy: 0.5740 - lr: 0.0010
Epoch 2/20
438/438 [==============================] - ETA: 0s - loss: 0.6576 - accuracy: 0.6280 ROC_test:0.721
 MicroF1_Test:0.66

Epoch 2: saving model to me_model_save/epo_02-accu_0.6602.hdf5
438/438 [==============================] - 2s 5ms/step - loss: 0.6576 - accuracy: 0.6280 - val_loss: 0.6470 - val_accuracy: 0.6602 - lr: 0.0010
Epoch 3/20
417/438 [===========================>..] - ETA: 0s - loss: 0.6328 - accuracy: 0.6586 ROC_test:0.72
 MicroF1_Test:0.646

Epoch 3: saving model to me_model_save/epo_03-accu_0.6462.hdf5

Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0009000000427477062.
438/438 [==============================] - 2s 6ms/st

In [21]:
%tensorboard --logdir N_logs/

Reusing TensorBoard on port 6006 (pid 3916), started 0:00:29 ago. (Use '!kill 3916' to kill it.)

<br>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>

In [22]:
kernel_init = GlorotNormal()

model = create_model('relu', kernel_init)

optimizer_ = optimizers.SGD(learning_rate = 0.001, momentum = 0.9, name = 'SGD')

model.compile(optimizer = optimizer_, loss = 'binary_crossentropy', metrics = ['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (Dense)         (None, 32)                96        
                                                                 
 dense1 (Dense)              (None, 32)                1056      
                                                                 
 dense2 (Dense)              (None, 32)                1056      
                                                                 
 dense3 (Dense)              (None, 32)                1056      
                                                                 
 dense4 (Dense)              (None, 32)                1056      
                                                                 
 dense5 (Dense)              (None, 32)                1056      
                                                                 
 output_layer (Dense)        (None, 1)                

In [23]:
# TensorBoard

log_dir = 'N_logs/4_relu_GlorotNormal'
tensorboard_ = TensorBoard(log_dir = log_dir, histogram_freq = 1, write_graph = True)

CallBack_list = [roc_score, f1_score_, terminateNaN, early_stopping, model_check_point,
                                                                     epoch_1, epoch_3, tensorboard_]

In [24]:
history = model.fit(x = x_train, y = y_train, epochs = EPOCH, verbose = 1,
            validation_data = (x_test, y_test), callbacks = CallBack_list)

Epoch 1/20
424/438 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5394 ROC_test:0.619
 MicroF1_Test:0.579

Epoch 1: saving model to me_model_save/epo_01-accu_0.5787.hdf5
438/438 [==============================] - 3s 5ms/step - loss: 0.6912 - accuracy: 0.5402 - val_loss: 0.6889 - val_accuracy: 0.5787 - lr: 0.0010
Epoch 2/20
428/438 [============================>.] - ETA: 0s - loss: 0.6864 - accuracy: 0.5845 ROC_test:0.652
 MicroF1_Test:0.607

Epoch 2: saving model to me_model_save/epo_02-accu_0.6072.hdf5
438/438 [==============================] - 2s 6ms/step - loss: 0.6864 - accuracy: 0.5843 - val_loss: 0.6830 - val_accuracy: 0.6072 - lr: 0.0010
Epoch 3/20
415/438 [===========================>..] - ETA: 0s - loss: 0.6805 - accuracy: 0.6052 ROC_test:0.669
 MicroF1_Test:0.617

Epoch 3: saving model to me_model_save/epo_03-accu_0.6173.hdf5
438/438 [==============================] - 2s 6ms/step - loss: 0.6802 - accuracy: 0.6048 - val_loss: 0.6754 - val_accuracy: 0.6

In [25]:
%tensorboard --logdir N_logs/

Reusing TensorBoard on port 6006 (pid 3916), started 0:00:54 ago. (Use '!kill 3916' to kill it.)

# Note 
Make sure that you are plotting tensorboard plots either in your notebook or you can try to create a pdf file with all the tensorboard screenshots.Please write your analysis of tensorboard results for each model.


# <font color='blue'>Comparision</font>

In [26]:
from prettytable import PrettyTable

x = PrettyTable()

x.field_names = ['Model', 'Activation Function', 'Kernal Initializer', 'No. of Epochs ran', 'Loss', 'Accuracy', 'ROC Test']
x.add_row([1, 'tanh', 'RandomUniform', 5, 0.6933, 0.5081, 0.504])
x.add_row([2, 'relu', 'RandomUniform', 3, 0.6932, 0.5006, 0.5])
x.add_row([3, 'relu', 'HeUniform', 8, 0.6011, 0.6776, 0.727])
x.add_row([4, 'relu', 'GlarotNormal', 10, 0.6011, 0.6701, 0.73])

print(x)

+-------+---------------------+--------------------+-------------------+--------+----------+----------+
| Model | Activation Function | Kernal Initializer | No. of Epochs ran |  Loss  | Accuracy | ROC Test |
+-------+---------------------+--------------------+-------------------+--------+----------+----------+
|   1   |         tanh        |   RandomUniform    |         5         | 0.6933 |  0.5081  |  0.504   |
|   2   |         relu        |   RandomUniform    |         3         | 0.6932 |  0.5006  |   0.5    |
|   3   |         relu        |     HeUniform      |         8         | 0.6011 |  0.6776  |  0.727   |
|   4   |         relu        |    GlarotNormal    |         10        | 0.6011 |  0.6701  |   0.73   |
+-------+---------------------+--------------------+-------------------+--------+----------+----------+


**<font color='red'>Observation</font>**

- This shows with activation function as `relu` and kernal initializer as `GlarotNormal` gives better **ROC Test** value than anu other models.
- Also this models runs for 10 epochs.

<br><br><br><br>

# <font color = 'red'>TensorBoard Outpots</font>

# <font color = 'blue'>Scalar</font>
<img src = 'Scalars.jpg'>

<br><br>

# <font color = 'blue'>Graphs</font>
<img src = 'https://i.imgur.com/BPlmgoE.png'>

<br><br>

# <font color = 'blue'>Distributions</font>
<img src = 'https://i.imgur.com/4dpHQf0.jpg'>

<br><br>

# <font color = 'blue'>Histograms</font>
<img src = 'https://i.imgur.com/WIWwXsr.jpg'>

<br><br>

# <font color = 'blue'>Time Series</font>
<img src = 'https://i.imgur.com/tO52xw1.jpg'>